In [1]:
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, InputLayer
import math

import os

Using TensorFlow backend.


In [2]:
def angleBetweenPoints(p0, p1, p2):
    a = (p1[0] - p0[0]) ** 2 + (p1[1] - p0[1]) ** 2
    b = (p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2
    c = (p2[0] - p0[0]) ** 2 + (p2[1] - p0[1]) ** 2
    return math.acos((a + b - c) / math.sqrt(4 * a * b))

class Agent:
    def __init__(self):
        self.featureExtractors = [getattr(self, m) for m in dir(self) if
                                  m.startswith("_Agent__extractFeature") and callable(getattr(self, m))]
        self.__reward=0
        self.__realStateDim = 82
        self.__stateDim = 6 + 8 * 6


    def __extractFeatureDistance(self, *args):
        "Euklidean distance from closest to (9,-1)"
        state = args[0].copy()
        xy = state[0, 2:].reshape(int((self.__realStateDim - 2) / 4), 4)[:,
             :2]  # lista punktów (x,y)(czubki segmentów macki)
        xy -= [9, -1]
        return np.sqrt(np.square(xy).sum(1).min())

    def __extractFeatureAngleParallelity(self, *args):
        state = args[0][0]
        "Angle stopping tentacle from parallelity with [(0,0),(9,-1)]"
        return angleBetweenPoints((9, -1), state[34:36], state[38:40])

    def __extractFeatureAngleParallelity2(self, *args):
        state = args[0][0]
        "Angle stopping tentacle from parallelity with [(0,0),(9,-1)]"
        return angleBetweenPoints((9, -1), state[74:76], state[78:80])

    def __extractFeatureVertexCloser(self, *args):
        "Which vertex of the tentacle is closer (-1 if lower, 1 if upper)"
        state = args[0].copy()
        xy = state[0, 2:].reshape(int((self.__realStateDim - 2) / 4), 4)[[9, 19],
             :2]  # lista punktów (x,y)(czubki segmentów macki)
        return np.argmin(np.square(xy - [9, -1]).sum(1)) * 2 - 1
    
    def __extractImportantPoints(self, *args):
        state = args[0][2:].copy()
        print(state)
        l = []
        for i in (0, 4, 7):
            l+=(state[4 * i:4 * i + 4])
            l+=(state[4 * i + 40:4 * i + 44])
        for i in range(6):
            l[i*4] -= 4.5
            l[i*4+1] -= 1
        return l

    def getFeatureVector(self, state, reward):
        "Convert input parameters to vecture of features"
        self.__reward += reward
        st=state.copy()
        state = np.array(list(state)).reshape((1, self.__realStateDim))
        f = []
        for m in self.featureExtractors:
            s=state.copy()
            # print(m.__name__,m(state,reward))
            f += [m(s, reward)]
        f+=self.__extractImportantPoints(st)
        return f

In [3]:
ag = Agent()

In [4]:
model = Sequential()
model.add(InputLayer(batch_input_shape=(1, 4 + 8 * 3)))
model.add(Dense(30, activation='sigmoid'))
model.add(Dense(20, activation='sigmoid'))
model.add(Dense(4 * 4, activation='linear'))
model.compile(loss='mse', optimizer='adam', metrics=['mae'])

model = keras.models.load_model('model33')

In [5]:
data = []
a = []
reward = []
for s in ("data001", "data098", "data150"):
    f = open(s, 'r')
    for i,line in enumerate(f):
        if i%3==0:
            data.append([float(x) for x in line.split(" ")])
        elif i%3==1:
            a.append([int(float(x)) for x in line.split(" ")])
        else:
            reward.append(float(line))

In [6]:
f=[]
anwser=[]
for i in range(len(data)):
    f.append(ag.getFeatureVector(data[i], reward[i]))
for i in range(len(a)):
    anwser.append(np.zeros((4*4)))
    x=a[i]
    w=0
    print()
    print(x)
    if x[1]==1:w+=2
    if x[2]==1: w+=1
    w*=4
    if x[19]==1:w+=2
    if x[20]==1: w+=1
    anwser[i][w] = 8 + 0.04*i
    print(w)
     
    

[0.9735005146973044, 1.021735404805655, -0.004487761759996865, 0.013535664666621318, 1.9986941893857613, 1.0040097845034655, -0.0009988938308323103, -0.0016771959466422952, 3.0022982287328412, 1.0015141506716545, 0.0014290133238283919, -0.0036057839902006485, 4.0012915706088865, 0.9998448459510588, 0.0007239083817972544, -0.004887049762076894, 4.999957937631293, 1.0003315114895313, -5.5092695986828385e-05, -0.005201187054413256, 5.99616455006271, 1.004795901417845, -0.0024245223014218364, -0.004973108796880163, 6.987427476770761, 1.0143237428631038, -0.0076635111074865505, -0.004436184666302155, 7.974002425878227, 1.0250391304674114, -0.012647194445004934, 0.00040435707561409587, 8.948682697443855, 1.0677082937347409, -0.014454526699670069, 0.011829224605838705, 10.046937431450853, 0.951320143988048, 0.00017921717032010932, -0.026905282967584432, 1.0742879529465712, -0.0010506680263928222, 0.037519818698121356, 0.003076148927465383, 2.0200315028618063, -0.0013056566196183756, 0.0126337

In [7]:
x = []
a=15
for i in range(2):
            y = a % 4
            for j in range((5, 5)[i]):
                x += {0: [0, 0, 0], 1: [0, 0, 1], 2: [0, 1, 0], 3: [0, 1, 1]}[y]
            a //= 4
print(x)
print(len(anwser))

[0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1]
46


In [8]:
batch_size = 1   # ile obrazków przetwarzamy na raz (aktualizacja wag sieci następuje raz na całą grupę obrazków)
epochs = 20         # ile epok będziemy uczyli

xx=np.array(f)
yy=np.array(anwser)
# Tutaj następuje właściwe uczenie
model.fit(xx, yy,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(xx, yy))

Train on 46 samples, validate on 46 samples
Epoch 1/20
46/46 [==============================] - 1s 13ms/step - loss: 3.0115 - mean_absolute_error: 0.6806 - val_loss: 2.2523 - val_mean_absolute_error: 0.5062
Epoch 2/20
46/46 [==============================] - 0s 6ms/step - loss: 1.7717 - mean_absolute_error: 0.4466 - val_loss: 1.3237 - val_mean_absolute_error: 0.3850
Epoch 3/20
46/46 [==============================] - 0s 6ms/step - loss: 1.0952 - mean_absolute_error: 0.3434 - val_loss: 0.8254 - val_mean_absolute_error: 0.2865
Epoch 4/20
46/46 [==============================] - 0s 6ms/step - loss: 0.6943 - mean_absolute_error: 0.2648 - val_loss: 0.5510 - val_mean_absolute_error: 0.2331
Epoch 5/20
46/46 [==============================] - 0s 6ms/step - loss: 0.4642 - mean_absolute_error: 0.2149 - val_loss: 0.3677 - val_mean_absolute_error: 0.1910
Epoch 6/20
46/46 [==============================] - 0s 6ms/step - loss: 0.3210 - mean_absolute_error: 0.1785 - val_loss: 0.2559 - val_mean_absolu

In [9]:
x=f[0]
print(len(x))
model.save("model33")

28
